In [ ]:
# Import dependencies
import pandas as pd
from sqlalchemy import create_engine

from config import PgAdmin_userid
# from config import PgAdmin_password

# Extract CSVs into DataFrames

In [ ]:
gdp_csv_file = "Resources/happiness_index.csv"
GDP_index_df = pd.read_csv(gdp_csv_file)
GDP_index_df

In [ ]:
happiness_csv_file = "Resources/happiness_index2019.csv"
happiness_index_df = pd.read_csv(happiness_csv_file)
happiness_index_df

In [ ]:
corruption_csv_file = "Resources/corruption_index.csv"
corruption_index_df = pd.read_csv(corruption_csv_file)
corruption_index_df

In [ ]:
population_csv_file = "Resources/population.csv"
population_index_df = pd.read_csv(population_csv_file)
population_index_df

# Transform to DataFrame

In [ ]:
GDP_data_df = GDP_index_df[['Country name', 'Logged GDP per capita']].copy()
GDP_data_df.head()

In [ ]:
GDP_data_df = GDP_data_df.rename(columns = {"Country name" : "Country",
                                         "Logged GDP per capita" : "GDP per capita"
                                         })
GDP_data_df.head()

In [ ]:
happiness_df = happiness_index_df[['Overall rank', "Country or region",  'Score']].copy()
happiness_df.head()

In [ ]:
happiness_df = happiness_df.rename(columns = {"Overall rank" : "Happiness Rank", 
                                              "Country or region" : "Country",
                                              "Score" : "Happiness Score"
                                         })
happiness_df.head()

In [ ]:
# new_df = GDP_data_df.merge(happiness_df , on ="Country", how = "inner")
# new_df

In [ ]:
merge_one = pd.merge(left = GDP_data_df, right = happiness_df, how = "inner", on = "Country" )
merge_one

In [ ]:
corruption_df = corruption_index_df[["Country (region)", "Ladder", "Corruption"]].copy()
corruption_df.head()

In [ ]:
corruption_df = corruption_df.rename(columns = {"Country (region)" : "Country", 
                                              "Ladder" : "Corruption Rank",
                                              "Corruption" : "Corruption Score"
                                         })
corruption_df.head()

In [ ]:
merge_two = pd.merge(left = merge_one, right = corruption_df, how = "inner", on = "Country" )
merge_two

In [ ]:
population_df = population_index_df[["Country (or dependency)", "Population (2020)", "Density (P/Km²)"]]
population_df.head()

In [ ]:
population_df  = population_df .rename(columns = {"Country (or dependency)" : "Country", 
                                              "Population (2020)" : "Population"
                                         })
population_df.head()

In [ ]:
combined_df = pd.merge(left = merge_two, right = population_df, how = "inner", on = "Country" )
combined_df

In [ ]:
new_combined_df = combined_df.reset_index()
new_combined_df["index"] += 1
new_combined_df = new_combined_df.rename(columns = {"index" : "Country ID"})
new_combined_df

In [ ]:
new_combined_df = new_combined_df.rename(columns = {"Country ID" : "country_id",
                                                   "Country" : "country_name",
                                                   "GDP per capita" : "gdp_per_capita",
                                                   "Happiness Rank" : "happiness_rank",
                                                   "Happiness Score" : "happiness_score",
                                                   "Corruption Rank" : "corruption_rank",
                                                   "Corruption Score" : "corruption_score",
                                                   "Population" : "population",
                                                   "Density (P/Km²)" : "density"
                                                   })


new_combined_df.head()

# Create database connection

In [ ]:
# connection_string = f"{PgAdmin_userid}:{PgAdmin_password}@localhost:5432/ETL_project_db"
connection_string = f"{PgAdmin_userid}@localhost:5432/ETL_project_db"
engine = create_engine(f'postgresql://{connection_string}')

In [ ]:
engine.table_names()

### Load DataFrames into database

In [ ]:
new_combined_df[["country_id", "country_name"]].to_sql(name="country", con = engine, if_exists="append", index=False)
    

In [ ]:
new_combined_df[["happiness_rank", "country_id", "happiness_score"]].to_sql(name="happiness_index", con = engine, if_exists="append", index=False)

In [ ]:
new_combined_df[["corruption_rank", "country_id", "corruption_score"]].to_sql(name="corruption_index", con = engine, if_exists="append", index=False)

In [ ]:
new_combined_df[["country_id", "population", "density"]].to_sql(name="population", con = engine, if_exists="append", index=False)

In [ ]:
new_combined_df[["country_id", "gdp_per_capita"]].to_sql(name="gdp_data", con = engine, if_exists="append", index=False)

